In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 64
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
        )
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
        )
        self.regr = nn.Sequential(
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(y, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [8]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [9]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_12_05_more_models'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_12_05_more_models-2024-12-06-15-42-17


In [10]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.02,   "lambda": 0.0,  'learning_rate': 5e-5},
    {"epochs": 2, "epsilon": 0.0075, "lambda": 0.15, 'learning_rate': 4e-5},
    {"epochs": 2, "epsilon": 0.004,  "lambda": 0.3,  'learning_rate': 3e-5},
    {"epochs": 2, "epsilon": 0.002,  "lambda": 0.4,  'learning_rate': 2e-5},
    {"epochs": 2, "epsilon": 0.0,    "lambda": 0.5,  'learning_rate': 1e-5}
]

In [11]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=100, context_size=200)
train_dataset.close()
val_dataset.close()

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.0271, 'grad_norm': 1.2878047227859497, 'learning_rate': 4.997222222222223e-05, 'epoch': 0.0}
{'loss': 0.0046, 'grad_norm': 0.0, 'learning_rate': 4.994444444444445e-05, 'epoch': 0.0}
{'loss': 0.0049, 'grad_norm': 0.10745882987976074, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.0}
{'loss': 0.0043, 'grad_norm': 0.10782333463430405, 'learning_rate': 4.9888888888888894e-05, 'epoch': 0.0}
{'loss': 0.0022, 'grad_norm': 0.034805770963430405, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.01}
{'loss': 0.0046, 'grad_norm': 0.10465945303440094, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 0.0035, 'grad_norm': 0.03636068105697632, 'learning_rate': 4.9805555555555554e-05, 'epoch': 0.01}
{'loss': 0.0034, 'grad_norm': 0.13969896733760834, 'learning_rate': 4.977777777777778e-05, 'epoch': 0.01}
{'loss': 0.0029, 'grad_norm': 0.03685876727104187, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 0.0058, 'grad_norm': 0.042724404484033585, 'learning_rate': 4.9722

  0%|          | 0/1000 [00:00<?, ?it/s]

{'eval_loss': 0.0011229101801291108, 'eval_runtime': 27.3769, 'eval_samples_per_second': 1461.086, 'eval_steps_per_second': 36.527, 'epoch': 1.0}
{'loss': 0.0014, 'grad_norm': 0.053860850632190704, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.0018, 'grad_norm': 0.17638817429542542, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.07176236063241959, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0018, 'grad_norm': 0.11345639079809189, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0009, 'grad_norm': 0.08604974299669266, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.0007, 'grad_norm': 0.10372030735015869, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0003, 'grad_norm': 0.0, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0009, 'grad_norm': 0.13673171401023865, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss': 0.0008, 'g

  0%|          | 0/1000 [00:00<?, ?it/s]

...

{'eval_loss': 0.003353700740262866, 'eval_runtime': 28.0878, 'eval_samples_per_second': 1424.108, 'eval_steps_per_second': 35.603, 'epoch': 1.0}
{'loss': 0.0035, 'grad_norm': 1.1253081560134888, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.0038, 'grad_norm': 0.4303174614906311, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0034, 'grad_norm': 0.5376290678977966, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0051, 'grad_norm': 0.3129482567310333, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0026, 'grad_norm': 0.26800772547721863, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.0028, 'grad_norm': 0.26594939827919006, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0021, 'grad_norm': 0.22119638323783875, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0025, 'grad_norm': 0.3098735213279724, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss': 0

  0%|          | 0/1000 [00:00<?, ?it/s]

{'eval_loss': 0.002900592051446438, 'eval_runtime': 27.7962, 'eval_samples_per_second': 1439.044, 'eval_steps_per_second': 35.976, 'epoch': 2.0}
{'train_runtime': 960.1417, 'train_samples_per_second': 749.889, 'train_steps_per_second': 18.747, 'train_loss': 0.003134594148542318, 'epoch': 2.0}


Processing models:   0%|          | 0/1 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [12]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)